In [1]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, datasets
from tensorflow.python.ops import math_ops


In [2]:
# Data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
num_samples = 5000
num_test_samples = 10000
train_images = train_images.astype('float32')[:num_samples]
test_images = test_images.astype('float32')
train_images /= 255.0
test_images /= 255.0
train_labels = keras.utils.to_categorical(train_labels, 10)[:num_samples]
test_labels = keras.utils.to_categorical(test_labels, 10)

batch_size = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(num_test_samples)


In [3]:


class MyModel(keras.Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.model = self.build_model()
        self.loss_tracker = keras.metrics.Mean(name='loss')
        self.acc_tracker = keras.metrics.Mean(name='acc')

    def build_model(self):
        input_tensor = layers.Input(shape=(32, 32, 3))
        x = layers.Conv2D(filters=32, kernel_size=(4, 4), activation='relu')(input_tensor)
        x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
        x = layers.Conv2D(filters=32, kernel_size=(4, 4), activation='relu')(x)
        x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
        x = layers.Conv2D(filters=64, kernel_size=(4, 4), activation='relu')(x)
        x = layers.Flatten()(x)
        output_tensor = layers.Dense(units=10, activation='softmax')(x)

        model = keras.Model(inputs=[input_tensor], outputs=[output_tensor])

        self.built = True
        return model

    def acc_fun(self, y_true, y_pred):
        y_true = math_ops.argmax(y_true, axis=-1)
        y_pred = math_ops.argmax(y_pred, axis=-1)
        return K.mean(math_ops.cast(math_ops.equal(y_true, y_pred), K.floatx()))

    @property
    def metrics(self):
        return [
            self.loss_tracker,
            self.acc_tracker,
        ]

    def test_step(self, test_data):
        x, y = test_data[0], test_data[1]
        y_pred = self.model(x)
        loss = keras.losses.CategoricalCrossentropy()(y, y_pred)
        acc = self.acc_fun(y, y_pred)
        return {'loss_test': loss, 'acc_test': acc}

    def train_step(self, data):
        x, y = data[0], data[1]
        with tf.GradientTape() as tape:
            y_pred = self.model(x)
            loss = keras.losses.CategoricalCrossentropy()(y, y_pred)
            acc = self.acc_fun(y, y_pred)
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state(acc)
        return {
            'loss': self.loss_tracker.result(),
            'acc_': self.acc_tracker.result()
        }



my_model = MyModel()
my_model.compile(optimizer='adam')

my_model.summary()

history = my_model.fit(
        train_dataset,
        # validation_data=(test_images, test_labels),
        validation_data=test_dataset,
        epochs=2,
        # batch_size=128,
        # callbacks=[callback],
    )


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 10)                56586     
                                                                 
Total params: 56,590
Trainable params: 56,586
Non-trainable params: 4
_________________________________________________________________
Epoch 1/2
40/40 [==============================] - 16s 365ms/step - loss: 2.1783 - acc_: 0.1518 - val_loss_test: 1.9886 - val_acc_test: 0.2565
Epoch 2/2
40/40 [==============================] - 11s 277ms/step - loss: 1.9095 - acc_: 0.2763 - val_loss_test: 1.8303 - val_acc_test: 0.3444
